<a href="https://colab.research.google.com/github/Adeyinka-hub/MY-MACHINE-LEARNING-AND-DEEP-LEARNING-WORKS-ON-KAGGLE-AND-ZINDI-RESPECTIVELY/blob/master/ZINDI_AIR_QUALITY_CHALLENGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install catboost

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import time

from sklearn import metrics
from sklearn.metrics import mean_squared_error,log_loss
from sklearn.model_selection import KFold,StratifiedKFold,GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
#import tqdm as tqdm
import xgboost as xgb
import lightgbm as lgb
from catboost import Pool,CatBoost
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_rows=500
pd.options.display.max_columns=100
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from scipy import stats
from scipy.stats import norm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
meta=pd.read_csv("/content/drive/My Drive/airqo_metadata.csv")
train=pd.read_csv("/content/drive/My Drive/train (1).csv")
test=pd.read_csv("/content/drive/My Drive/test.csv")
sub_df=pd.read_csv("/content/drive/My Drive/sample_submission.csv")

In [ ]:
meta.drop("Unnamed: 0",axis=1,inplace=True)

In [ ]:
cols=['dist_trunk','dist_primary','dist_secondary','dist_motorway']
for col in cols:
    meta[col +"_greater_than_5k?"]=np.where(np.isnan(meta[col].values), 1, 0)

In [ ]:
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    x["var_"+col_name]=x[col_name].apply(np.var)
    x["median_"+col_name]=x[col_name].apply(np.median)
    x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    return x

In [ ]:
features=[x for x in meta.columns if x not in ['location','dist_motorway_greater_than_5k','dist_motorway']]
for col_name in features:
    meta=aggregate_features(meta,col_name)

In [ ]:
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
for feature in features : 
    train[feature]=train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])
    test[feature]=test[feature].apply(lambda x: [ replace_nan(X)  for X in x.replace("nan"," ").split(",")])    

In [ ]:
for df in [train,test]:
    for x in range(121):
        df['temp_'+ str(x)]=df.temp.str[x]
        df['precip_'+ str(x)]=df.precip.str[x]
        df['rel_humidity_'+ str(x)]=df.rel_humidity.str[x]
        #df['wind_dir_'+ str(x)]=df.wind_dir.str[x]
        #df['wind_spd_'+ str(x)]=df.wind_spd.str[x]
        df['atmos_press_'+ str(x)]=df.atmos_press.str[x]
        

In [ ]:
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    x["var_"+col_name]=x[col_name].apply(np.var)
    x["median_"+col_name]=x[col_name].apply(np.median)
    x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    return x  
def remove_nan_values(x):
    return [e for e in x if not math.isnan(e)]


In [ ]:
train=pd.merge(train,meta,on='location',how='left')

In [ ]:
test=pd.merge(test,meta,on='location',how='left')

In [ ]:
from tqdm import tqdm

In [ ]:
split=train.shape[0]
data=pd.concat([train,test],sort=False).reset_index(drop=True)

In [ ]:
import math
for col_name in tqdm(features):
    data[col_name]=data[col_name].apply(remove_nan_values)

100%|██████████| 6/6 [00:02<00:00,  2.37it/s]


In [ ]:
for col_name in tqdm(features):
    data=aggregate_features(data,col_name)

100%|██████████| 6/6 [00:17<00:00,  2.99s/it]


In [ ]:
data.drop(features,1,inplace=True)

In [ ]:
le=LabelEncoder()
data['location']=le.fit_transform(data['location'])

In [ ]:
train_df=data[:split]
test_df=data[split:]

In [ ]:
pip install datasist

In [ ]:
for df in [train_df,test_df]:
    df['tem_range']=df['max_temp']-df['min_temp']
    df['prec_range']=df['max_precip']-df['min_precip']
    df['real_hum_range']=df['max_rel_humidity']-df['min_rel_humidity']
    df['range_wind_dir']=df['max_wind_dir']-df['min_wind_dir']
    df['range_wind_spd']=df['max_wind_spd']-df['min_wind_spd']
    df['rng_atmos_press']=df['max_atmos_press']-df['min_atmos_press']

In [ ]:
train_df.shape, test_df.shape

((15539, 680), (5035, 680))

In [ ]:
train_df.head()

,ID,location,target,temp_0,precip_0,rel_humidity_0,atmos_press_0,temp_1,precip_1,rel_humidity_1,atmos_press_1,temp_2,precip_2,rel_humidity_2,atmos_press_2,temp_3,precip_3,rel_humidity_3,atmos_press_3,temp_4,precip_4,rel_humidity_4,atmos_press_4,temp_5,precip_5,rel_humidity_5,atmos_press_5,temp_6,precip_6,rel_humidity_6,atmos_press_6,temp_7,precip_7,rel_humidity_7,atmos_press_7,temp_8,precip_8,rel_humidity_8,atmos_press_8,temp_9,precip_9,rel_humidity_9,atmos_press_9,temp_10,precip_10,rel_humidity_10,atmos_press_10,temp_11,precip_11,rel_humidity_11,...,median_dist_motorway_greater_than_5k?,ptp_dist_motorway_greater_than_5k?,max_temp,min_temp,mean_temp,std_temp,var_temp,median_temp,ptp_temp,max_precip,min_precip,mean_precip,std_precip,var_precip,median_precip,ptp_precip,max_rel_humidity,min_rel_humidity,mean_rel_humidity,std_rel_humidity,var_rel_humidity,median_rel_humidity,ptp_rel_humidity,max_wind_dir,min_wind_dir,mean_wind_dir,std_wind_dir,var_wind_dir,median_wind_dir,ptp_wind_dir,max_wind_spd,min_wind_spd,mean_wind_spd,std_wind_spd,var_wind_spd,median_wind_spd,ptp_wind_spd,max_atmos_press,min_atmos_press,mean_atmos_press,std_atmos_press,var_atmos_press,median_atmos_press,ptp_atmos_press,tem_range,prec_range,real_hum_range,range_wind_dir,range_wind_spd,rng_atmos_press
0,ID_train_0,2,45.126304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0,27.208333,19.275000,22.299527,2.594011,6.728895,21.358333,7.933333,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.891500,0.510818,0.758530,0.120234,0.014456,0.806292,0.380682,345.726333,6.535299,209.574655,120.231802,14455.686315,220.039988,339.191034,1.517500,0.162500,0.677348,0.394041,0.155268,0.666250,1.355000,87.871667,87.614167,87.752434,0.070374,0.004952,87.762083,0.257500,7.933333,0.000,0.380682,339.191034,1.355000,0.257500
1,ID_train_1,3,79.131702,22.533333,0.102,0.744583,90.320000,21.716667,0.0,0.808083,90.377500,20.833333,0.0,0.911167,90.440833,20.983333,0.000,0.916333,90.472500,20.875000,0.000,0.929750,90.454167,20.141667,0.0,0.954000,90.394167,19.375000,0.0,0.964083,90.333333,18.791667,0.034,0.970750,90.310833,18.775000,0.017,0.964917,90.343333,18.641667,0.017,0.963833,90.363333,18.558333,0.017,0.968333,90.365833,18.533333,0.017,0.966417,...,1.0,0,33.616667,17.983333,24.679063,4.266955,18.206903,23.791667,15.633333,0.561,0.0,0.007025,0.051706,0.002673,0.0,0.561,0.979250,0.403083,0.758058,0.160952,0.025906,0.791000,0.576167,357.730286,3.543198,177.109855,96.802929,9370.807133,153.164843,354.187088,3.009167,0.248333,0.899208,0.539199,0.290736,0.744167,2.760833,90.725000,90.056667,90.429924,0.156000,0.024336,90.429167,0.668333,15.633333,0.561,0.576167,354.187088,2.760833,0.668333
2,ID_train_10,0,32.661304,28.975000,0.000,0.573333,88.551667,27.950000,0.0,0.597167,88.464167,29.600000,0.0,0.566833,88.319167,26.425000,0.102,0.627667,88.240000,22.091667,0.136,0.755417,88.230000,21.775000,0.0,0.777417,88.268333,22.333333,0.0,0.787000,88.370000,21.950000,2.160,0.831917,88.520000,20.441667,1.276,0.880083,88.563333,20.441667,0.051,0.882500,88.645000,20.950000,0.936,0.848917,88.696667,19.800000,0.612,0.906000,...,1.0,0,31.841667,18.458333,24.112317,3.776377,14.261020,23.304167,13.383333,7.804,0.0,0.151767,0.798401,0.637445,0.0,7.804,0.930333,0.376750,0.712944,0.168129,0.028267,0.766458,0.553583,357.265773,0.581510,201.373211,92.105195,8483.367004,208.901301,356.684263,2.751667,0.331667,1.038656,0.585221,0.342484,0.942500,2.420000,88.813333,87.982500,88.420931,0.179481,0.032213,88.425000,0.830833,13.383333,7.804,0.553583,356.684263,2.420000,0.830833
3,ID_train_100,0,53.850238,22.966667,0.000,0.843083,88.615000,24.266667,0.0,0.790250,88.530833,25.275000,0.0,0.737500,88.400000,25.625000,0.000,0.728000,88.271667,25.866667,0.000,0.704917,88.207500,25.091667,0.0,0.741167,88.178333,24.025000,0.0,0.764250,88.206667,22.425000,0.000,0.850500,88.280833,2

In [ ]:
test_df.head()

,ID,location,target,temp_0,precip_0,rel_humidity_0,atmos_press_0,temp_1,precip_1,rel_humidity_1,atmos_press_1,temp_2,precip_2,rel_humidity_2,atmos_press_2,temp_3,precip_3,rel_humidity_3,atmos_press_3,temp_4,precip_4,rel_humidity_4,atmos_press_4,temp_5,precip_5,rel_humidity_5,atmos_press_5,temp_6,precip_6,rel_humidity_6,atmos_press_6,temp_7,precip_7,rel_humidity_7,atmos_press_7,temp_8,precip_8,rel_humidity_8,atmos_press_8,temp_9,precip_9,rel_humidity_9,atmos_press_9,temp_10,precip_10,rel_humidity_10,atmos_press_10,temp_11,precip_11,rel_humidity_11,...,median_dist_motorway_greater_than_5k?,ptp_dist_motorway_greater_than_5k?,max_temp,min_temp,mean_temp,std_temp,var_temp,median_temp,ptp_temp,max_precip,min_precip,mean_precip,std_precip,var_precip,median_precip,ptp_precip,max_rel_humidity,min_rel_humidity,mean_rel_humidity,std_rel_humidity,var_rel_humidity,median_rel_humidity,ptp_rel_humidity,max_wind_dir,min_wind_dir,mean_wind_dir,std_wind_dir,var_wind_dir,median_wind_dir,ptp_wind_dir,max_wind_spd,min_wind_spd,mean_wind_spd,std_wind_spd,var_wind_spd,median_wind_spd,ptp_wind_spd,max_atmos_press,min_atmos_press,mean_atmos_press,std_atmos_press,var_atmos_press,median_atmos_press,ptp_atmos_press,tem_range,prec_range,real_hum_range,range_wind_dir,range_wind_spd,rng_atmos_press
15539,ID_test_0,2,NaN,23.966667,0.0,0.689833,87.643333,22.883333,0.0,0.722333,87.670000,22.283333,0.0,0.775583,87.705833,21.483333,0.0,0.838417,87.750833,20.908333,0.0,0.847500,87.791667,19.908333,0.0,0.859500,87.793333,19.141667,0.0,0.889917,87.761667,18.775000,0.0,0.918500,87.779167,18.533333,0.000,0.948833,87.775000,18.558333,0.0,0.965417,87.702500,18.200000,0.0,0.979583,87.721667,17.766667,0.0,0.993917,...,1.0,0,26.858333,17.050000,21.389532,2.731833,7.462909,20.908333,9.808333,5.728,0.0,0.204165,0.880254,0.774846,0.0,5.728,0.999833,0.586083,0.842947,0.124179,0.015420,0.859250,0.413750,357.223551,7.092226,231.104760,83.856214,7031.864555,217.738239,350.131326,1.915833,0.171667,0.812555,0.392783,0.154279,0.767500,1.744167,88.020833,87.474167,87.748506,0.132764,0.017626,87.752500,0.546667,9.808333,5.728,0.413750,350.131326,1.744167,0.546667
15540,ID_test_1,2,NaN,18.991667,0.0,0.999750,87.892500,19.708333,0.0,0.997250,87.956667,20.958333,0.0,0.956333,87.992500,22.783333,0.0,0.851583,87.992500,24.325000,0.0,0.777583,87.980833,25.016667,0.0,0.735833,87.928333,25.541667,0.0,0.704333,87.826667,26.925000,0.0,0.612583,87.728333,26.991667,0.000,0.583167,87.630833,25.941667,0.0,0.699417,87.572500,24.366667,0.0,0.769333,87.662500,23.525000,0.0,0.749000,...,1.0,0,26.991667,17.275000,21.302824,2.289268,5.240748,20.716667,9.716667,15.831,0.0,0.208719,1.517969,2.304229,0.0,15.831,0.999750,0.583167,0.830610,0.106384,0.011318,0.852000,0.416583,351.333059,6.290985,212.924101,91.254295,8327.346424,181.691165,345.042074,2.099167,0.210000,0.852562,0.456733,0.208605,0.771667,1.889167,88.181667,87.572500,87.936963,0.113756,0.012940,87.928333,0.609167,9.716667,15.831,0.416583,345.042074,1.889167,0.609167
15541,ID_test_10,3,NaN,25.116667,0.0,0.760167,90.340000,25.083333,0.0,0.733583,90.297500,24.408333,0.0,0.784750,90.305000,23.575000,0.0,0.825833,90.355833,22.350000,0.0,0.869750,90.442500,21.525000,0.0,0.906500,90.493333,21.266667,0.0,0.902667,90.536667,20.433333,0.0,0.890667,90.571667,19.800000,0.000,0.895333,90.607500,19.983333,0.0,0.886750,90.538333,19.575000,0.0,0.918500,90.509167,19.341667,0.0,0.926333,...,1.0,0,29.850000,17.108333,22.144421,2.943047,8.661528,21.458333,12.741667,1.327,0.0,0.040190,0.164596,0.027092,0.0,1.327,0.986750,0.515167,0.868258,0.096427,0.009298,0.903750,0.471583,356.508481,2.279896,167.731590,98.024235,9608.750685,144.225803,354.228585,3.127500,0.237500,0.803140,0.569737,0.324600,0.567500,2.890000,90.785000,90.115000,90.521357,0.136316,0.018582,90.536667,0.670000,12.741667,1.327,0.471583,354.228585,2.890000,0.670000
15542,ID_test_100,2,NaN,18.716667,0.0,0.951417,87.988333,20.608333,0.0,0.925750,88.063333,22.283333,0.0,0.860333,88.078333,23.983333,0.0,0.7

In [ ]:
import datasist as ds
train_df = ds.feature_engineering.drop_redundant(train_df)
test_df = ds.feature_engineering.drop_redundant(test_df)

Dropped ['dist_motorway', 'dist_motorway_greater_than_5k?', 'std_loc_altitude', 'var_loc_altitude', 'ptp_loc_altitude', 'std_km2', 'var_km2', 'ptp_km2', 'std_aspect', 'var_aspect', 'ptp_aspect', 'std_dist_tertiary', 'var_dist_tertiary', 'ptp_dist_tertiary', 'std_dist_unclassified', 'var_dist_unclassified', 'ptp_dist_unclassified', 'std_dist_residential', 'var_dist_residential', 'ptp_dist_residential', 'std_popn', 'var_popn', 'ptp_popn', 'std_hh', 'var_hh', 'ptp_hh', 'std_hh_cook_charcoal', 'var_hh_cook_charcoal', 'ptp_hh_cook_charcoal', 'std_hh_cook_firewood', 'var_hh_cook_firewood', 'ptp_hh_cook_firewood', 'std_hh_burn_waste', 'var_hh_burn_waste', 'ptp_hh_burn_waste', 'std_dist_trunk_greater_than_5k?', 'var_dist_trunk_greater_than_5k?', 'ptp_dist_trunk_greater_than_5k?', 'std_dist_primary_greater_than_5k?', 'var_dist_primary_greater_than_5k?', 'ptp_dist_primary_greater_than_5k?', 'std_dist_secondary_greater_than_5k?', 'var_dist_secondary_greater_than_5k?', 'ptp_dist_secondary_greater_

In [ ]:
train_df.shape, test_df.shape

((15539, 627), (5035, 626))

In [ ]:
train_target = train['target'].values.astype(np.float32)

In [ ]:
param = {'num_leaves': 200,
     'min_data_in_leaf': 70, 
     'objective':'regression',
     'max_depth': 19,
     'learning_rate': 0.005,
     "boosting": "gbdt",
     "feature_fraction": 0.2,
     "bagging_freq": 0,
     "bagging_fraction": 1.0 ,
     "bagging_seed": 4,
     "metric": 'rmse',
     "verbosity": 1,
     "verbose_eval": False
    }


from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import xgboost as xgb
import time

features = [c for c in train_df.columns if c not in ['ID','target']]
target = train_df['target']

folds = KFold(n_splits=5, shuffle=True, random_state=15)#15
oof_lgb = np.zeros(len(train_df))
predictions_lgb = np.zeros(len(test_df))
start = time.time()
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])

    num_round = 10900
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
    oof_lgb[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    predictions_lgb += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits
oof_lgb[oof_lgb < 0] = 2
print("CV score: {:<8.5f}".format(mean_squared_error(oof_lgb, target)**0.5))

fold n°0
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 34.7697	valid_1's rmse: 34.5148
[200]	training's rmse: 29.9452	valid_1's rmse: 31.229
[300]	training's rmse: 26.8021	valid_1's rmse: 29.331
[400]	training's rmse: 24.5905	valid_1's rmse: 28.1806
[500]	training's rmse: 22.9397	valid_1's rmse: 27.3701
[600]	training's rmse: 21.6222	valid_1's rmse: 26.8078
[700]	training's rmse: 20.5645	valid_1's rmse: 26.4029
[800]	training's rmse: 19.6809	valid_1's rmse: 26.0759
[900]	training's rmse: 18.9337	valid_1's rmse: 25.8259
[1000]	training's rmse: 18.2914	valid_1's rmse: 25.6234
[1100]	training's rmse: 17.7399	valid_1's rmse: 25.4684
[1200]	training's rmse: 17.2564	valid_1's rmse: 25.336
[1300]	training's rmse: 16.8123	valid_1's rmse: 25.2341
[1400]	training's rmse: 16.4005	valid_1's rmse: 25.1421
[1500]	training's rmse: 16.0232	valid_1's rmse: 25.0571
[1600]	training's rmse: 15.6766	valid_1's rmse: 24.9848
[1700]	training's rmse: 15.3357	valid_1's rm

In [ ]:
sub=test_df[["ID"]]
sub['target'] = predictions_lgb
sub.to_csv('sub_addtional102.csv', index = False)   #CV score: 23.58731